In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from copy import deepcopy

import matplotlib.pyplot as plt
import seaborn as sns

from calcmodule import MeanAbsError, Correlation, MeanError, mMAE, BiasCorrection
from calcmodule import cohen_d, LMEM, TukeyHSD
from scipy.stats import ttest_ind, f_oneway

from statsmodels.stats.multicomp import pairwise_tukeyhsd


import warnings
warnings.filterwarnings("ignore")

In [ ]:
BASE = "/NFS/Users/moonsh/thesis/result/"
SAVEPATH = '/NFS/Users/moonsh/thesis/result/merge/'
PHENOPATH = '/NFS/Users/moonsh/thesis/data/'
analysis_type = ['train-test', 'longitudinal', 'BAG',] # test-retest
model_type = ['Center', "Local", "FedAvg", "FedProx", "MOON"]

data_name = ['HCP', 'NKI-RK', 'UCLA-CNP', 'OAS3', 'JBNU', 'SALD', 'CamCAN', 'DecNef', 'DLBS', 'IXI']
ind_data_name = ['ADNI', 'COBRE', 'MCIC', 'NUSDAST', 'OAS4', 'OAS2', 'SLIM', 'CoRR', 'fcon1000']

total_data_name = data_name + ind_data_name

# ADNI, COBRE, MCIC, NUSDAST, OAS4: Utility
# OAS2, SLIM: Consistency
# CoRR, fcon1000: Generalizabilitiy

patient_data = ["ADNI", "COBRE", "MCIC", "NUSDAST", "OAS4"]
pin_col = ["Age", "Sex(1=m,2=f)", "ImageFile"]

## Merge Result DataFrame

In [ ]:
# for _type in model_type:
#     print(_type)
#     _type_path = BASE+"original"+f"/{_type}/"
#     # _type_path = BASE+f"/{_type}/"
#     wandb_proj_name_list = os.listdir(_type_path)
#     print(wandb_proj_name_list)

#     for d_name in ['OAS2', "SLIM"]:
#         merge_df = pd.DataFrame()

#         for rp_idx, wandb_proj_name in enumerate(wandb_proj_name_list):
#             rp_idx += 1
#             pwd = _type_path + "/" + wandb_proj_name
#             df_list = os.listdir(pwd)
            
#             for df_name in df_list:
#                 if d_name in df_name:
#                     df = pd.read_csv(pwd + "/" + df_name)
#                     df.drop_duplicates(subset='Subject', keep='first', inplace=True)
#                     df.sort_values(by='Subject', inplace=True)

#                     if rp_idx == 1:
#                         merge_df['Subject'] = df['Subject']
#                         merge_df['Age'] = df['Age']

#                     merge_df[f"pred_{rp_idx}"] = df['pred_age']

#                     if rp_idx == 5:
#                         merge_df['avg_pred_age'] = merge_df[[f"pred_{i+1}" for i in range(5)]].mean(axis=1)
#                         merge_df['mode'] = df['mode']                        

#                         if d_name in patient_data:
#                             pheno_df = pd.read_csv(os.path.join(PHENOPATH, d_name, f"{d_name}_Phenotype_total.csv"))
#                             pheno_col = pheno_df.columns
#                             filtered_col = [col for col in pheno_col if col not in pin_col]
#                             pheno_df = pheno_df[filtered_col]

#                             merge_df = pd.merge(merge_df, pheno_df, on='Subject', how='inner')
#                     break
#         folder = os.path.join(SAVEPATH, f"{d_name}")
        
#         if not os.path.exists(folder):
#             os.makedirs(folder)

#         merge_df.to_csv(os.path.join(SAVEPATH, folder, f"{d_name}_{_type}_follow.csv"), index=False)

## Train-Test Performance

In [ ]:
MERGEBASE = "/NFS/Users/moonsh/thesis/result/merge/"

In [ ]:
performance_mae_df = pd.DataFrame(columns=['Dataset', 'Center', 'Local', 'FedAvg', 'FedProx', 'MOON'])
performance_r2_df = pd.DataFrame(columns=['Dataset', 'Center', 'Local', 'FedAvg', 'FedProx', 'MOON'])

mixed_performance_mae_df = pd.DataFrame(columns=['Dataset', 'Center', 'Local', 'FedAvg', 'FedProx', 'MOON'])
mixed_performance_r2_df = pd.DataFrame(columns=['Dataset', 'Center', 'Local', 'FedAvg', 'FedProx', 'MOON'])

df_per_dataset = []

fig, axs = plt.subplots(2, 5, figsize=(28, 11))

for idx, name in enumerate(data_name):

    temp_model_df = pd.DataFrame()

    ax = axs[idx//5, idx%5]
    TEMPPATH = os.path.join(MERGEBASE, name)

    new_row_mae = {'Dataset': name}
    new_row_r2 = {'Dataset': name}

    mixed_row_mae = {'Dataset': name}
    mixed_row_r2 = {'Dataset': name}

    viz_df = pd.DataFrame(columns=['Dataset', 'Center', 'Local', 'FedAvg', 'FedProx', 'MOON'])

    for _type in model_type: # ['Center', "Local", "FedAvg", "FedProx", "MOON"]
        temp_save_df = pd.DataFrame()

        df = pd.read_csv(os.path.join(TEMPPATH, f"{name}_{_type}.csv"))

        temp_save_df['Model'] = [_type]*len(df)
        temp_save_df['AE'] = np.abs(df['Age'] - df['avg_pred_age'])
        temp_save_df = pd.concat([temp_save_df, df], axis=1)

        test_df = df[df['mode'] == 'Test']
        temp_mae_list = []
        temp_r2_list = []
        temp_df = pd.DataFrame()

        for pred_idx in range(1, 6):
            temp_mae = MeanAbsError(test_df, 'Age', f"pred_{pred_idx}")
            temp_r2 = Correlation(test_df, 'Age', f"pred_{pred_idx}", method='r2')
            temp_mae_list.extend([temp_mae])
            temp_r2_list.extend([temp_r2])
        
        
        mae_mean = np.mean(temp_mae_list)
        mae_std = np.std(temp_mae_list)

        r2_mean = np.mean(temp_r2_list)
        r2_std = np.std(temp_r2_list)

        new_row_mae.update({_type: f"{mae_mean:.3f}±{mae_std:.3f}"})
        new_row_r2.update({_type: f"{r2_mean:.3f}±{r2_std:.3f}"})

        mixed_MAE = MeanAbsError(test_df, 'Age', "avg_pred_age")
        mixed_r2 = Correlation(test_df, 'Age', "avg_pred_age", method='r2')

        mixed_row_mae.update({_type: f"{mixed_MAE:.3f}"})
        mixed_row_r2.update({_type: f"{mixed_r2:.3f}"})

        temp_df['model'] = [_type]*5
        temp_df['mae'] = temp_mae_list
        temp_df['r2'] = temp_r2_list

        viz_df = pd.concat([viz_df, temp_df], axis=0)

        temp_model_df = pd.concat([temp_model_df, temp_save_df], axis=0)
    
    df_per_dataset.append(temp_model_df)
    sns.boxplot(data=viz_df, x="model", y="mae", width=0.6, palette="Set3", ax=ax)

    ax.set_title(f"[{name}] MAE Performance", fontsize=14)
    ax.set_xlabel("Model", fontsize=12)
    ax.set_ylabel("MAE", fontsize=12)

    if name == 'CamCAN':
        ax.set_ylim(2, 11.5)
    elif name == 'DecNef' or name == 'IXI':
        ax.set_ylim(3.5, 7)

    else:
        ax.set_ylim(2, 7)
         
    mixed_performance_mae_df = pd.concat([mixed_performance_mae_df, pd.DataFrame(mixed_row_mae, index=[0])], axis=0)
    mixed_performance_r2_df = pd.concat([mixed_performance_r2_df, pd.DataFrame(mixed_row_r2, index=[0])], axis=0)

    performance_mae_df = pd.concat([performance_mae_df, pd.DataFrame(new_row_mae, index=[0])], axis=0)
    performance_r2_df = pd.concat([performance_r2_df, pd.DataFrame(new_row_r2, index=[0])], axis=0)

# plt.savefig(os.path.join('/NFS/Users/moonsh/thesis/asset', "MAE_Performance.png"), dpi=600, bbox_inches='tight', format='png', pad_inches=0.2)
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(28, 11))

colors = sns.color_palette("Set3", n_colors=3)

axs = axs.flatten()

for i in range(len(mixed_performance_mae_df)):

    ax = axs[i]
    
    # 데이터 추출
    dataset_name = mixed_performance_mae_df.iloc[i, 0]
    center_perform = mixed_performance_mae_df.iloc[i, 1]
    local_perform = mixed_performance_mae_df.iloc[i, 2]
    fedavg_perform = mixed_performance_mae_df.iloc[i, 3]
    fedprox_perform = mixed_performance_mae_df.iloc[i, 4]
    moon_perform = mixed_performance_mae_df.iloc[i, 5]


    perform_bar = [float(center_perform), float(local_perform), float(fedavg_perform), float(fedprox_perform), float(moon_perform)]

    ax.bar([0, 1, 2], perform_bar[2:], color=colors)

    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['FedAvg', 'FedProx', 'MOON'])
    ax.set_ylim(1, np.array(perform_bar).max()*1.1)

    ax.axhline(float(center_perform), color='purple', linestyle='-', linewidth=2, label='Center')
    ax.axhline(float(local_perform), color='navy', linestyle='--', linewidth=2, label='Local')

    ax.set_title(dataset_name, fontsize=16)
    ax.set_xlabel("Model", fontsize=12)
    ax.set_ylabel("Performance", fontsize=12)
    ax.legend(fontsize=15, loc='upper right')


# plt.savefig('/NFS/Users/moonsh/thesis/asset/mixed_MAE_Performance.png', dpi=600, bbox_inches='tight', format='png', pad_inches=0.2)
plt.tight_layout()
plt.show()

In [ ]:
# performance_mae_df.to_csv(os.path.join(MERGEBASE, "performance_mae.csv"), index=False, encoding='utf-8')
# performance_r2_df.to_csv(os.path.join(MERGEBASE, "performance_r2.csv"), index=False, encoding='utf-8')

# mixed_performance_mae_df.to_csv(os.path.join(MERGEBASE, "mixed_performance_mae.csv"), index=False)
# mixed_performance_r2_df.to_csv(os.path.join(MERGEBASE, "mixed_performance_r2.csv"), index=False)

In [ ]:
stat_list = []

for data_idx in range(len(df_per_dataset)):
    df = df_per_dataset[data_idx]
    df = df[df['mode'] == 'Test']

    _, new_df = LMEM(df, fixed_cols='Model', random_cols='Subject', dependent_col='AE', intercept="Local", verbose=False)
    stat_list.append(new_df)

In [ ]:
stat_list = []

for data_idx in range(len(df_per_dataset)):
    df = df_per_dataset[data_idx]
    df = df[df['mode'] == 'Test']

    new_df = TukeyHSD(df, fixed_col='Model', dependent_col='AE', verbose=False)
    stat_list.append(new_df)

In [ ]:
stat_list[2]

## Independent DataFrame

In [ ]:
MERGEBASE = "/NFS/Users/moonsh/thesis/result/merge/"

In [ ]:
performance_mae_df = pd.DataFrame(columns=['Dataset', 'Center','FedAvg', 'FedProx', 'MOON'])
performance_r2_df = pd.DataFrame(columns=['Dataset', 'Center', 'FedAvg', 'FedProx', 'MOON'])

mixed_performance_mae_df = pd.DataFrame(columns=['Dataset', 'Center', 'FedAvg', 'FedProx', 'MOON'])
mixed_performance_r2_df = pd.DataFrame(columns=['Dataset', 'Center', 'FedAvg', 'FedProx', 'MOON'])

df_per_dataset = []

for name in ind_data_name:
    temp_model_df = pd.DataFrame()
    TEMPPATH = os.path.join(MERGEBASE, name)

    new_row_mae = {'Dataset': name}
    new_row_r2 = {'Dataset': name}

    mixed_row_mae = {'Dataset': name}
    mixed_row_r2 = {'Dataset': name}

    for _type in ['Center', "FedAvg", "FedProx", "MOON"]:

        temp_save_df = pd.DataFrame()

        if name == 'OAS2' or name == 'SLIM':
            df = pd.read_csv(os.path.join(TEMPPATH, f"{name}_{_type}_base.csv"))
        
        else:
            df = pd.read_csv(os.path.join(TEMPPATH, f"{name}_{_type}.csv"))
        # corr_df = BiasCorrection(df, 'Age', 'avg_pred_age', method='linear', base_df=df[df['mode'] == 'Train'])

        # test_df = df[df['mode'] == 'Test']

        temp_save_df['Model'] = [_type]*len(df)
        temp_save_df['AE'] = np.abs(df['Age'] - df['avg_pred_age'])
        temp_save_df = pd.concat([temp_save_df, df], axis=1)


        test_df = df.copy()
        if "Control" in test_df.columns:
            test_df = test_df[test_df['Control']=='HC']
        temp_mae_list = []
        temp_r2_list = []

        for pred_idx in range(1, 6):
            temp_mae = MeanAbsError(test_df, 'Age', f"pred_{pred_idx}")
            temp_r2 = Correlation(test_df, 'Age', f"pred_{pred_idx}", method='r2')
            temp_mae_list.extend([temp_mae])
            temp_r2_list.extend([temp_r2])
        
        mae_mean = np.mean(temp_mae_list)
        mae_std = np.std(temp_mae_list)

        r2_mean = np.mean(temp_r2_list)
        r2_std = np.std(temp_r2_list)

        new_row_mae.update({_type: f"{mae_mean:.3f}±{mae_std:.3f}"})
        new_row_r2.update({_type: f"{r2_mean:.3f}±{r2_std:.3f}"})

        mixed_MAE = MeanAbsError(test_df, 'Age', "avg_pred_age")
        mixed_r2 = Correlation(test_df, 'Age', "avg_pred_age", method='r2')

        mixed_row_mae.update({_type: f"{mixed_MAE:.3f}"})
        mixed_row_r2.update({_type: f"{mixed_r2:.3f}"})

        temp_model_df = pd.concat([temp_model_df, temp_save_df], axis=0)

    df_per_dataset.append(temp_model_df)

    mixed_performance_mae_df = pd.concat([mixed_performance_mae_df, pd.DataFrame(mixed_row_mae, index=[0])], axis=0)
    mixed_performance_r2_df = pd.concat([mixed_performance_r2_df, pd.DataFrame(mixed_row_r2, index=[0])], axis=0)

    performance_mae_df = pd.concat([performance_mae_df, pd.DataFrame(new_row_mae, index=[0])], axis=0)
    performance_r2_df = pd.concat([performance_r2_df, pd.DataFrame(new_row_r2, index=[0])], axis=0)

In [ ]:
performance_mae_df

In [ ]:
# performance_mae_df.to_csv(os.path.join(MERGEBASE, "ind_performance_mae.csv"), index=False)
# performance_r2_df.to_csv(os.path.join(MERGEBASE, "ind_performance_r2.csv"), index=False)

# mixed_performance_mae_df.to_csv(os.path.join(MERGEBASE, "ind_mixed_performance_mae.csv"), index=False)
# mixed_performance_r2_df.to_csv(os.path.join(MERGEBASE, "ind_mixed_performance_r2.csv"), index=False)

In [ ]:
mixed_performance_mae_df

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(15, 15))

colors = sns.color_palette("Set3", n_colors=3)

axs = axs.flatten()

for i in range(len(mixed_performance_mae_df)):

    ax = axs[i]
    
    # 데이터 추출
    dataset_name = mixed_performance_mae_df.iloc[i, 0]
    center_perform = mixed_performance_mae_df.iloc[i, 1]
    fedavg_perform = mixed_performance_mae_df.iloc[i, 2]
    fedprox_perform = mixed_performance_mae_df.iloc[i, 3]
    moon_perform = mixed_performance_mae_df.iloc[i, 4]


    perform_bar = [float(center_perform), float(fedavg_perform), float(fedprox_perform), float(moon_perform)]

    ax.bar([0, 1, 2], perform_bar[1:], color=colors)

    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['FedAvg', 'FedProx', 'MOON'])
    ax.set_ylim(2, np.array(perform_bar).max()*1.1)

    ax.axhline(float(center_perform), color='purple', linestyle='-', linewidth=2, label='Center')

    ax.set_title(dataset_name, fontsize=14)
    # ax.set_xlabel("Model", fontsize=12)
    ax.set_ylabel("Performance", fontsize=12)
    ax.legend(fontsize=13, loc='upper right')


plt.savefig('/NFS/Users/moonsh/thesis/asset/ind_mixed_MAE_Performance.png', dpi=600, bbox_inches='tight', format='png', pad_inches=0.2)
plt.tight_layout()
plt.show()

In [ ]:
stat_list = []

for data_idx in range(len(df_per_dataset)):
    df = df_per_dataset[data_idx]
    # df = df[df['mode'] == 'Test']

    _, new_df = LMEM(df, fixed_cols='Model', random_cols='Subject', dependent_col='AE', intercept="Center", verbose=False)
    stat_list.append(new_df)

In [ ]:
stat_list = []

for data_idx in range(len(df_per_dataset)):
    df = df_per_dataset[data_idx]
    # df = df[df['mode'] == 'Test']

    new_df = TukeyHSD(df, fixed_col='Model', dependent_col='AE', verbose=False)
    stat_list.append(new_df)

## Patient Analysis

In [ ]:
MERGEBASE = "/NFS/Users/moonsh/thesis/result/merge/"
patient_data_name = ['ADNI', 'OAS4', 'COBRE', 'MCIC', 'NUSDAST']

In [ ]:
result_list = []
fig, axs = plt.subplots(1, 5, figsize=(35, 6))


for idx, name in enumerate(['ADNI', 'OAS4', 'COBRE', 'MCIC', 'NUSDAST', ]):
    ax = axs[idx]
    
    TEMPPATH = os.path.join(MERGEBASE, name)

    data_df = pd.DataFrame()
    
    for _type in ['Center', "FedAvg", "FedProx", "MOON"]:
        df = pd.read_csv(os.path.join(TEMPPATH, f"{name}_{_type}.csv"))
        df = df[["Subject", "Age", "avg_pred_age", "Control"]]

        if len(df['Control'].unique()) == 3:
            df["Control"] = pd.Categorical(df["Control"], categories=["HC", "MCI", "AD"], ordered=True)
            
        elif len(df['Control'].unique()) == 2:
            df["Control"] = pd.Categorical(df["Control"], categories=["HC", "SCZ"], ordered=True)
        
        corr_df = BiasCorrection(df, 'Age', 'avg_pred_age', method='linear',)
        corr_df['model'] = [_type]*len(corr_df)

        data_df = pd.concat([data_df, corr_df], axis=0)

        # result_dict = {name: {'Model': _type, 'df': corr_df}}
        # result_list.append(result_dict)
    result_list.append(data_df)


    sns.boxplot(data=data_df, x="model", y="corr_PAD", hue="Control", width=0.6, palette="Set3", ax=ax)

    # 그래프 설정
    ax.set_title(f"[{name}] Brin Delta", fontsize=22)
    ax.set_xlabel("Model", fontsize=15)
    ax.set_ylabel("Corr. Delta", fontsize=15)
    ax.legend(title="Control", fontsize=14, loc='lower right')


# plt.savefig(os.path.join('/NFS/Users/moonsh/thesis/asset', "Patient_Brin_Delta.png"), dpi=600, bbox_inches='tight', format='png', pad_inches=0.2)
plt.tight_layout()
plt.show()
    

In [ ]:
for name, i in zip(patient_data_name, range(len(result_list))):
    total_df = result_list[i]

    for _type in ['Center', "FedAvg", "FedProx", "MOON"]:
        print(f"[{name}] {_type}")
        df = total_df[total_df['model']==_type]

        if len(df['Control'].unique()) == 3:
            HC = df[df['Control']=='HC']['corr_PAD']
            MCI = df[df['Control']=='MCI']['corr_PAD']
            AD = df[df['Control']=='AD']['corr_PAD']
            stat1, p1 = ttest_ind(HC, MCI, equal_var=True)
            stat2, p2 = ttest_ind(HC, AD, equal_var=True)

            stat1, p1 = round(stat1, 3), round(p1, 3)
            stat2, p2 = round(stat2, 3), round(p2, 3)

            print(f"{name}: HC vs MCI - {stat1}, {p1}")
            print(f"{name}: HC vs AD - {stat2}, {p2}")
            
        else:
            HC = df[df['Control']=='HC']['corr_PAD']
            SCZ = df[df['Control']=='SCZ']['corr_PAD']
            stat, p = ttest_ind(HC, SCZ, equal_var=True)

            stat, p = round(stat, 3), round(p, 3)

            print(f"{name}: HC vs SCZ - {stat}, {p}")
            
    print("----------------------------------------------")

In [ ]:
for name, i in zip(patient_data_name, range(len(result_list))):
    total_df = result_list[i]

    for _type in ['Center', "FedAvg", "FedProx", "MOON"]:
        print(f"[{name}] {_type}")
        df = total_df[total_df['model']==_type]
        if len(df['Control'].unique()) == 3:
            HC = df[df['Control']=='HC']['corr_PAD']
            MCI = df[df['Control']=='MCI']['corr_PAD']
            AD = df[df['Control']=='AD']['corr_PAD']
            stat1, p1 = f_oneway(HC, MCI, AD)
            stat1, p1 = round(stat1, 3), round(p1, 3)

            print(f"{name}: ANOVA - {stat1}, {p1}")
            
        else:
            HC = df[df['Control']=='HC']['corr_PAD']
            SCZ = df[df['Control']=='SCZ']['corr_PAD']
            stat, p = f_oneway(HC, SCZ,)

            stat, p = round(stat, 3), round(p, 3)

            print(f"{name}: ANOVA - {stat}, {p}")
    print("----------------------------------------------")   

In [ ]:
for name, i in zip(patient_data_name, range(len(result_list))):
    total_df = result_list[i]

    for _type in ['Center', "FedAvg", "FedProx", "MOON"]:
        print(f"[{name}] {_type}")
        df = total_df[total_df['model']==_type]
        if len(df['Control'].unique()) == 3:
            HC = list(df[df['Control']=='HC']['corr_PAD'])
            MCI = list(df[df['Control']=='MCI']['corr_PAD'])
            AD = list(df[df['Control']=='AD']['corr_PAD'])

            data = pd.DataFrame({
                                    'value': HC + MCI + AD,
                                    'group': ['HC'] * len(HC) + ['MCI'] * len(MCI) + ['AD'] * len(AD)
                                })
            
            tukey = pairwise_tukeyhsd(data['value'], data['group'], alpha=0.05)
            print(tukey)
            
        else:
            HC = list(df[df['Control']=='HC']['corr_PAD'])
            SCZ = list(df[df['Control']=='SCZ']['corr_PAD'])

            data = pd.DataFrame({
                                    'value': HC + SCZ,
                                    'group': ['HC'] * len(HC) + ['SCZ'] * len(SCZ)
                                })
            
            tukey = pairwise_tukeyhsd(data['value'], data['group'], alpha=0.05)
            print(tukey)

    print("----------------------------------------------")

## Association